# Setup inicial

## Importação de bibliotecas:

In [39]:
# Instalação das bibliotecas necessárias
!pip install keras-tuner
!pip install pandas tensorflow scikit-learn
!pip install gdown

# Importação de bibliotecas
import kerastuner as kt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import metrics
from keras.utils import split_dataset
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.metrics import AUC

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Importação dos dados:

In [40]:
# importação dos dados:

!gdown --id 1nriPPuYUMXeB6BkCjz_bQI_45WZfxViC

df = pd.read_csv("creditcard.csv")

C:\Users\patri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\gdown\__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1nriPPuYUMXeB6BkCjz_bQI_45WZfxViC
From (redirected): https://drive.google.com/uc?id=1nriPPuYUMXeB6BkCjz_bQI_45WZfxViC&confirm=t&uuid=34a12757-5fdf-4363-96e6-5e0896ae2e90
To: c:\Users\patri\Downloads\creditcard.csv

  0%|          | 0.00/151M [00:00<?, ?B/s]
  0%|          | 524k/151M [00:00<01:13, 2.05MB/s]
  1%|▏         | 2.10M/151M [00:00<00:22, 6.75MB/s]
  3%|▎         | 4.72M/151M [00:00<00:13, 10.6MB/s]
  5%|▍         | 7.34M/151M [00:00<00:09, 14.8MB/s]
  6%|▋         | 9.44M/151M [00:00<00:10, 13.3MB/s]
  7%|▋         | 11.0M/151M [00:01<00:13, 10.3MB/s]
  9%|▊         | 13.1M/151M [00:01<00:15, 8.86MB

## Análise do conjunto de dados:

Abaixo e possivel visalizar o dataframe importado:

In [41]:
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

Apenas ao visualizar os dados diretamente, sem entender sua origem não é possível ter certeza o que significa cada coluna, mas ainda sim é possível realizar algumas observações:

As colunas de 0 a 29 são referentes ao input para o modelo, sendo todas do tipo float.

Dentre essas colunas é possível observar colunas sendo representadas por V1-V28, além de uma coluna "Time" possivelmente relacionada à tempo e uma "Amount" relacionado a quantidade, mas sem ter conhecimento sobre a origem dos dados não é possível ter uma ideia clara de que cada coluna significa. Apenas que os dados são referentes a utilização e fraudes em cartão de crédito.


In [43]:
df["Class"].unique()

array([0, 1], dtype=int64)

Além disso, também é possível observar que os dados possuem uma coluna "Classe" do tipo int, referente a classificação de fraudes no cartão de crédito. As quais podem assumir os valores 0 ou 1, provavelmente realizado uma classificação binária de fraude ou não.


## Criação do modelo inicial:

Abaixo será criado a primeira versão do modelo de rede neural:

In [44]:
#Criação do modelo de rede neural inicial
model = keras.Sequential([
    layers.InputLayer(input_shape=(30,)),
    layers.Dense(32, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid"),
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=['accuracy', metrics.F1Score()],
)


C:\Users\patri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


## Treinamento do modelo inicial:

Primeiramente para evitar qualquer enviesamento do modelo os conjuntos de treino e teste serão separados, sendo 20% para teste e 80% para treino:

In [45]:
# Limitando tamanho do dataset para rodar mais rapidamente o modelo
df = df.sample(n=10000, random_state=42)

# Separando a variável alvo
X = df.drop('Class', axis=1)
y = df['Class']

# Realizando o balanceamento das classes
df_balanced = pd.concat([X, y], axis=1)

df_minority = df_balanced[df_balanced['Class'] == 1]
df_majority = df_balanced[df_balanced['Class'] == 0]

df_minority_upsampled = resample(df_minority,
                                 replace=True,
                                 n_samples=len(df_majority),
                                 random_state=42)

df_balanced = pd.concat([df_majority, df_minority_upsampled])

X_balanced = df_balanced.drop('Class', axis=1)
y_balanced = df_balanced['Class']

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

### Resultados:

In [46]:
# Realizando treinamento do modelo
model.fit(X_train, y_train, epochs=5, batch_size=30, validation_data=(X_test, y_test))

Epoch 1/5
533/533 ━━━━━━━━━━━━━━━━━━━━ 1s 911us/step - accuracy: 0.5247 - f1_score: 0.5204 - loss: 818.8192 - val_accuracy: 0.6863 - val_f1_score: 0.6375 - val_loss: 13.5361
Epoch 2/5
533/533 ━━━━━━━━━━━━━━━━━━━━ 0s 689us/step - accuracy: 0.6262 - f1_score: 0.6488 - loss: 30.2652 - val_accuracy: 0.4970 - val_f1_score: 0.6593 - val_loss: 22.1011
Epoch 3/5
533/533 ━━━━━━━━━━━━━━━━━━━━ 0s 679us/step - accuracy: 0.6787 - f1_score: 0.6796 - loss: 21.7105 - val_accuracy: 0.8190 - val_f1_score: 0.7911 - val_loss: 8.4702
Epoch 4/5
533/533 ━━━━━━━━━━━━━━━━━━━━ 0s 698us/step - accuracy: 0.7107 - f1_score: 0.7046 - loss: 20.1325 - val_accuracy: 0.5013 - val_f1_score: 0.6597 - val_loss: 31.0393
Epoch 5/5
533/533 ━━━━━━━━━━━━━━━━━━━━ 0s 698us/step - accuracy: 0.7296 - f1_score: 0.7248 - loss: 16.5331 - val_accuracy: 0.6612 - val_f1_score: 0.5307 - val_loss: 35.4821


Abaixo é possível ver os resultados obtidos pelo modelo base:

In [47]:
# Fazendo previsões no conjunto de teste
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Calculando as métricas
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred)

# pritando as metricas
print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print(f"AUC-ROC: {auc_roc}")


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 581us/step
Accuracy: 0.6612418627941913
Recall: 0.3115757266700663
F1-score: 0.4745631067961165
AUC-ROC: 0.6550500374619392


## otimização de hiperparâmetros:

Abaixo será utilizado o método de random search para testar quais hiperparâmetros tem o melhor impacto no modelo, para isso serão testadas as funções de ativação relu sigmoid e adam para camada oculta e os otimizadores adam, sgd, rmsprop, nadam e lion para o modelo.

In [53]:
# Definindo funções de ativação e otimizadores a serem testados com o random search
activation_functions = ['relu', 'sigmoid', 'tanh']
optimizers = ['adam', 'sgd', 'rmsprop', 'nadam', 'lion']

# Criação de modelo base para os teste
def create_model(activation, optimizer):
    model = models.Sequential([
        layers.InputLayer(input_shape=(30,)),
        layers.Dense(32, activation=activation),
        layers.Dense(16, activation=activation),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', AUC()],
    )
    return model

# Criando lista para salvar os resultados
results = []
num_trials = 15

# utilizando random search para testar combinações
for _ in range(num_trials):
    activation = np.random.choice(activation_functions)
    optimizer = np.random.choice(optimizers)

    # realizando teste da combinacao
    model = create_model(activation, optimizer)
    history = model.fit(X_train, y_train, epochs=5, batch_size=30, validation_data=(X_test, y_test), verbose=0)

    # Armazenar o resultado
    val_accuracy = history.history['val_accuracy'][-1]
    results.append((activation, optimizer, val_accuracy))

# obtendo a melhor combinação
best_result = max(results, key=lambda x: x[2])
best_activation, best_optimizer, best_accuracy = best_result

# Printando a melhor combinação
print(f'Melhor função de ativação: {best_activation}')
print(f'Melhor otimizador: {best_optimizer}')

C:\Users\patri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
C:\Users\patri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
C:\Users\patri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
C:\Users\patri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: 

Melhor função de ativação: relu
Melhor otimizador: lion


## Resultados:

Abaixo é possível visualizar os resultados do modelo otimizado:

In [56]:
# recuperando o modelo com a melhor combinação de hiperparâmetros
best_model = create_model(best_activation, best_optimizer)
best_model.fit(X_train, y_train, epochs=5, batch_size=20, validation_data=(X_test, y_test))

# Fazer previsões no conjunto de teste
y_pred = best_model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Calculando as métricas
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred)

# Imprimindo as métricas
print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print(f"AUC-ROC: {auc_roc}")


Epoch 1/5


C:\Users\patri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 792us/step - accuracy: 0.5230 - auc_104: 0.5232 - loss: 1021.0737 - val_accuracy: 0.6074 - val_auc_104: 0.5991 - val_loss: 100.6241
Epoch 2/5
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 619us/step - accuracy: 0.5595 - auc_104: 0.5625 - loss: 68.1079 - val_accuracy: 0.7001 - val_auc_104: 0.6922 - val_loss: 20.9823
Epoch 3/5
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 622us/step - accuracy: 0.6001 - auc_104: 0.6062 - loss: 24.6473 - val_accuracy: 0.7894 - val_auc_104: 0.8442 - val_loss: 5.0462
Epoch 4/5
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 624us/step - accuracy: 0.6550 - auc_104: 0.6891 - loss: 10.7102 - val_accuracy: 0.8348 - val_auc_104: 0.8935 - val_loss: 1.8929
Epoch 5/5
799/799 ━━━━━━━━━━━━━━━━━━━━ 1s 617us/step - accuracy: 0.6735 - auc_104: 0.7132 - loss: 5.0637 - val_accuracy: 0.8573 - val_auc_104: 0.8941 - val_loss: 0.7811
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step
Accuracy: 0.85728592889334
Recall: 0.7511473737888832
F1-score: 0.8378839590443686
AUC-ROC: 0.855406446363207


## Conclusão:

Após otimizar os hiperparâmetros do modelo usando o método de random search, onde foram escolhidos a função de ativação ReLU e o otimizador Lion, os resultados melhoraram consideravelmente em comparação com o modelo original.

Comparação dos Resultados:

Modelo Original:

* Accuracy: 66.12%
* Recall: 31.16%
* F1-score: 47.46%
* AUC-ROC: 65.51%

Modelo Otimizado:

* Accuracy: 85.73%
* Recall: 75.11%
* F1-score: 83.79%
* AUC-ROC: 85.54%

Análise:

Accuracy aumentou de 66.12% para 85.73%, mostrando que o modelo otimizado acerta mais previsões no geral.

Recall subiu de 31.16% para 75.11%, indicando que o modelo otimizado é muito melhor em identificar corretamente os casos positivos.

F1-score melhorou de 47.46% para 83.79%, equilibrando melhor a precisão e o recall.

AUC-ROC subiu de 65.51% para 85.54%, indicando que o modelo otimizado distingue melhor entre as classes.

Conclusão:

A otimização dos hiperparâmetros foi uma tática eficaz, resultando em um modelo com desempenho  superior em todas as metricas. Fazendo com que ao realizar otimização para ReLU e Lion houvessem avanços significativos no modelo.
